In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
!wc -l ../fhvhv_tripdata_2021-01.csv

 11908469 ../fhvhv_tripdata_2021-01.csv


In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('../fhvhv_tripdata_2021-01.csv')

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [8]:

df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!head -n 101 ../fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [11]:
!wc -l head.csv

     101 head.csv


In [12]:
import pandas as pd

df_pandas = pd.read_csv('head.csv')

df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [14]:
spark.createDataFrame(df_pandas).schema

/Users/saschapogadaev/miniconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [16]:
from pyspark.sql import types

In [17]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True) 
])

In [19]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('../fhvhv_tripdata_2021-01.csv')
    

In [21]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

break large file into smaller one

In [22]:
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [23]:
df.write.parquet('fhvhv/2021/01')

24/03/03 17:41:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [24]:
df = spark.read.parquet('fhvhv/2021/01')

In [32]:
from pyspark.sql import functions as F

In [35]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    else:
        return f'e/{num:03x}'

In [36]:
crazy_stuf_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [38]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuf_udf(df.dispatching_base_num)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID', 'base_id') \
    .show()

+-----------+------------+------------+------------+-------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|base_id|
+-----------+------------+------------+------------+-------+
| 2021-01-17|  2021-01-17|         148|         138|  e/b30|
| 2021-01-17|  2021-01-17|         231|          79|  s/b13|
| 2021-01-17|  2021-01-17|          79|         138|  s/b13|
| 2021-01-17|  2021-01-17|          14|         228|  e/acc|
| 2021-01-17|  2021-01-17|          73|         192|  e/acc|
| 2021-01-17|  2021-01-17|           7|         129|  e/acc|
| 2021-01-17|  2021-01-17|         129|          61|  e/acc|
| 2021-01-17|  2021-01-17|          21|          14|  e/b47|
| 2021-01-17|  2021-01-17|          14|          40|  e/b47|
| 2021-01-17|  2021-01-17|          42|         142|  e/a7a|
| 2021-01-17|  2021-01-17|         238|         250|  e/a7a|
| 2021-01-17|  2021-01-17|         225|          61|  e/9ce|
| 2021-01-17|  2021-01-17|         225|         132|  e/9ce|
| 2021-01-17|  2021-01-1

In [30]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df.hvfhs_license_num == 'HV0003').take(5)

[Row(pickup_datetime=datetime.datetime(2021, 1, 17, 6, 20, 53), dropoff_datetime=datetime.datetime(2021, 1, 17, 6, 45, 39), PULocationID=148, DOLocationID=138),
 Row(pickup_datetime=datetime.datetime(2021, 1, 17, 6, 27, 43), dropoff_datetime=datetime.datetime(2021, 1, 17, 6, 39, 8), PULocationID=231, DOLocationID=79),
 Row(pickup_datetime=datetime.datetime(2021, 1, 17, 6, 44, 48), dropoff_datetime=datetime.datetime(2021, 1, 17, 7, 4, 54), PULocationID=79, DOLocationID=138),
 Row(pickup_datetime=datetime.datetime(2021, 1, 17, 6, 48, 16), dropoff_datetime=datetime.datetime(2021, 1, 17, 6, 56, 14), PULocationID=14, DOLocationID=228),
 Row(pickup_datetime=datetime.datetime(2021, 1, 17, 6, 8, 5), dropoff_datetime=datetime.datetime(2021, 1, 17, 6, 14, 56), PULocationID=73, DOLocationID=192)]